# LangChain 入門


In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env", override=True)

## Model


### Model の基本的な使い方


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！私はジョンと言います"),
    AIMessage("こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage("私の名前がわかりますか？"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

### ストリーミング


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("こんにちは！"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

## Prompt template


### ChatPromptTemplate


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "料理名：{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "カレー"})
for message in prompt_value.messages:
    print(f"{message.type}: {message.content}")

### （参考）f-string や string.format()での実装


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

dish = "カレー"
messages = [
    SystemMessage(content="ユーザーが入力した料理のレシピを考えてください。"),
    HumanMessage(content=f"料理名：{dish}"),
]

for message in messages:
    print(f"{message.type}: {message.content}")

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage


def create_messages(dish: str) -> list:
    return [
        SystemMessage(content="ユーザーが入力した料理のレシピを考えてください。"),
        HumanMessage(content="料理名：{dish}".format(dish=dish)),
    ]


messages = create_messages("カレー")
for message in messages:
    print(f"{message.type}: {message.content}")

## with_structured_output


### with_structured_output を使った構造化出力


In [ ]:
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)
structured_model = model.with_structured_output(Recipe)
result = structured_model.invoke("カレーのレシピを教えて")

print(type(result))
print(result)

### 参考）LCEL（LangChain Expression Language）


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

chain = prompt | model.with_structured_output(Recipe)
result = chain.invoke({"dish": "カレー"})

print(result)

# Weave のセットアップとトレースの確認


In [ ]:
# 実行する前に、Notebook上で「Restart」を実行してください

from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.env", override=True)

# APIキーの先頭3文字が表示されれば、環境変数に設定できています
print(os.environ["WANDB_API_KEY"][:3])
assert os.environ["WANDB_API_KEY"]

In [ ]:
# Weaveの初期化
import weave

weave.init("training-llm-app")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

prompt_value = prompt.invoke({"dish": "カレー"})
ai_message = model.invoke(prompt_value)
print(ai_message.content)

# ここでWeaveのトレースを確認してください

# RAG (検索拡張生成) の基礎


## LangChain での RAG の実装をステップバイステップで動かそう


### Document loader


In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    path="../tmp/langchain-docs/src/langsmith/",
    glob="**/*.mdx",
    loader_cls=TextLoader,
)

raw_docs = loader.load()
print(len(raw_docs))

### Document transformer


In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

### Embedding model


In [ ]:
from langchain.embeddings import init_embeddings

embeddings = init_embeddings(model="text-embedding-3-small", provider="openai")

In [ ]:
query = "LangSmithのトレース機能について教えて"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

### Vector store


In [ ]:
# 注意:
# このセルの処理は、大勢が同時に実行するとレートリミットのエラーになる可能性があります
# もしもエラーになった場合は、少し時間をおいてもう一度実行してください

from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)
vector_store.reset_collection()
vector_store.add_documents(docs)


### Retriever


In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
query = "LangSmithのトレース機能について教えて"

context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

for i, doc in enumerate(context_docs):
    print(
        "-" * 10 + f" {i + 1}/{len(context_docs)}: {doc.metadata['source']} " + "-" * 10
    )
    print(doc.page_content)


### LangChain を使った RAG の実装


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

prompt = ChatPromptTemplate.from_template('''
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = init_chat_model(
    model="gpt-5-nano",
    model_provider="openai",
    reasoning_effort="minimal",
)

In [ ]:
import weave


@weave.op
def invoke_rag(query: str) -> str:
    documents = retriever.invoke(query)
    prompt_value = prompt.invoke({"question": query, "context": documents})
    ai_message = model.invoke(prompt_value)
    return ai_message.content


query = "LangSmithのトレース機能について教えて"
output = invoke_rag(query)
print(output)